In [3]:
# For modules loading
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import json
from tqdm import tqdm
import regex as re
from nltk.corpus import stopwords
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
#from transformers import AutoTokenizer, T5ForConditionalGeneration
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM, T5ForConditionalGeneration
import torch
from itertools import cycle
import os 
from transformers import pipeline

from rouge_score import rouge_scorer

# For BM25
import src.bm25_IR

/Users/adamwagnerhoegh/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Loading devset

In [7]:
dev_set = pd.read_csv('devset/dev_set_w_IR.csv')

In [8]:
dev_set

,question,answer,text,pnumber,law number,tf_idf_k1,tf_idf_k2,tf_idf_k3,bm25_k1,bm25_k2,bm25_k3,dense_cls_k1,dense_cls_k2,dense_cls_k3,dense_max_k1,dense_max_k2,dense_max_k3,dense_mean_k1,dense_mean_k2,dense_mean_k3
0,"Hvad har ejeren af en ejerlejlighed, sammen me...","Grunden, fælles bestanddele og tilbehør.",Ejeren af en ejerlejlighed har sammen med andr...,3,LOV nr 908 af 18/06/2020,Ejeren af en ejerlejlighed har sammen med andr...,Hver ejerlejlighed anses som en selvstændig fa...,I denne lov forstås ved: Ejerlejlighed: En lej...,Ejeren af en ejerlejlighed har sammen med andr...,Ejeren af en ejerlejlighed har sammen med andr...,Ejeren af en ejerlejlighed har sammen med andr...,Hver ejerlejlighed anses som en selvstændig fa...,Hver ejerlejlighed anses som en selvstændig fa...,Hver ejerlejlighed anses som en selvstændig fa...,"Det kan aftales, at lejeren af et blandet leje...","Det kan aftales, at lejeren af et blandet leje...","Det kan aftales, at lejeren af et blandet leje...",Ved skriftlig budgivning opfordrer fogeden sky...,Ved skriftlig budgivning opfordrer fogeden sky...,Ved skriftlig budgivning opfordrer fogeden sky...
1,Hvem fastsætter eller aftaler bestemmelser om ...,Finansministeren fastsætter eller aftaler best...,Højskolen skal følge de af finansministeren fa...,30,LBK nr 780 af 08/08/2019,Højskolen skal følge de af finansministeren fa...,Institutionen skal følge de af finansministere...,"For forstandere, viceforstandere og lærere ska...",Højskolen skal følge de af finansministeren fa...,Højskolen skal følge de af finansministeren fa...,Højskolen skal følge de af finansministeren fa...,Professionshøjskolens navn fastsættes i vedtæg...,Professionshøjskolens navn fastsættes i vedtæg...,Professionshøjskolens navn fastsættes i vedtæg...,Virksomheden kan efter finansministerens nærme...,Virksomheden kan efter finansministerens nærme...,Virksomheden kan efter finansministerens nærme...,Told- og skatteforvaltningen kan fastsætte reg...,Told- og skatteforvaltningen kan fastsætte reg...,Told- og skatteforvaltningen kan fastsætte reg...
2,Hvad skal Beskæftigelsesministeriet og Finanst...,Den indsendte årsrapport skal i det mindste in...,Uden ugrundet ophold efter repræsentantskabets...,25 l,LBK nr 1110 af 10/10/2014,Uden ugrundet ophold efter bestyrelsens godken...,Uden ugrundet ophold efter bestyrelsens godken...,Uden ugrundet ophold efter repræsentantskabets...,Uden ugrundet ophold efter bestyrelsens godken...,Uden ugrundet ophold efter bestyrelsens godken...,Uden ugrundet ophold efter bestyrelsens godken...,Arbejdsmarkedets Erhvervssikring betaler afgif...,Arbejdsmarkedets Erhvervssikring betaler afgif...,Arbejdsmarkedets Erhvervssikring betaler afgif...,Den godkendte årsrapport skal indsendes til Er...,Den godkendte årsrapport skal indsendes til Er...,Den godkendte årsrapport skal indsendes til Er...,"§ 10 finder ikke anvendelse for gaver, der er ...","§ 10 finder ikke anvendelse for gaver, der er ...","§ 10 finder ikke anvendelse for gaver, der er ..."
3,Hvor mange procent må kapitalandele i og lån y...,Kapitalandele i og lån ydet til en virksomhed ...,Følgende grænser for Arbejdsmarkedets Tillægsp...,26 e,LBK nr 1110 af 10/10/2014,Følgende grænser for Lønmodtagernes Dyrtidsfon...,Følgende grænser for Arbejdsmarkedets Tillægsp...,Følgende grænser for Arbejdsmarkedets Erhvervs...,Følgende grænser for Lønmodtagernes Dyrtidsfon...,Følgende grænser for Lønmodtagernes Dyrtidsfon...,Følgende grænser for Lønmodtagernes Dyrtidsfon...,Et pengeinstituts kapitalandele erhvervet for ...,Et pengeinstituts kapitalandele erhvervet for ...,Et pengeinstituts kapitalandele erhvervet for ...,Bestyrelsen i et e-pengeinstitut eller betalin...,Bestyrelsen i et e-pengeinstitut eller betalin...,Bestyrelsen i et e-pengeinstitut eller betalin...,Skadesforsikringsselskaberne skal efter hvert ...,Skadesforsikringsselskaberne skal efter hvert ...,Skadesforsikringsselskaberne skal efter hvert ...
4,Hvad er en betingelse for retten til jobpræmie?,D

### KennethTM/gpt-neo-1.3B-danish

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the model and tokenizer
MODEL_NAME = "KennethTM/gpt-neo-1.3B-danish"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

# Set the device
DEVICE = "mps" if torch.backends.mps.is_available() else "cpu" # set up for mac here, change to cuda if needed
model.to(DEVICE)

In [ ]:
neo_answers_tf_idf_k1 = []
# evaluating tf-idf
for question, documents in tqdm(zip(dev_set['question'], dev_set['tf_idf_k1']), desc='Answering questions'):

    # assemble a prompt from the documents, question and prompting an answer
    prompt = f"Relevante paragraffer: {documents} Spørgsmål: {question} \nIndsæt svar her baseret på de relevante paragraffer:"

    # tokenize
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(DEVICE)

    max_length = len(input_ids[0]) + 100

    # generate an answer within torch.no_grad() to save compute
    with torch.no_grad():
        outputs = model.generate(
            input_ids,
            max_length=max_length,
            pad_token_id=tokenizer.eos_token_id,
            # generation set to stop at '.' as it otherwise just repeats itself (think it's because we don't sample)
            eos_token_id=tokenizer.encode(' Spørgsmål')[0]
        )

    # decode generated answer
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True).strip(' Spørgsmål')

    # append answer to list
    neo_answers_tf_idf_k1.append(answer[len(prompt):].strip())  # strip the prompt to leave just the answer

### strombergnlp/dant5-large

In [ ]:
# Load the pretrained T5 model and tokenizer
model_name = "strombergnlp/dant5-large"  
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
DEVICE = "mps" if torch.backends.mps.is_available() else "cpu" # set up for mac here, change to cuda if needed
model.to(DEVICE)


/Users/adamwagnerhoegh/miniconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


T5ForConditionalGeneration(
  (shared): Embedding(32128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=1024, out_features=4096, bias=False)
              (wo): Linear(in_features=4096, out_features=1024, bias=False)
              (d

In [18]:
#for question, documents in tqdm(zip(dev_set['question, str'], dev_set['tf_idf_k1']), desc='Answering questions'):

# Example question and context

idx = 10

question = dev_set.loc[idx, 'question']
documents = dev_set.loc[idx, 'dense_cls_k1']

# Format the input for T5
input_text = f"Relevante paragraffer: {documents}\nSpørgsmål: {question}\nIndsæt svar her baseret på de relevante paragraffer:"

# Tokenize the input and generate an answer
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(DEVICE)

max_length = len(input_ids[0]) + 100

with torch.no_grad():
    outputs = model.generate(
        input_ids,
        max_length=max_length,
        pad_token_id=tokenizer.eos_token_id,
        # generation set to stop at ' Spørgsmål' as it otherwise just repeats itself (think it's because we don't sample)
        eos_token_id=tokenizer.encode(' Spørgsmål')[0]
    )

# Decode and print the generated answer
answer = tokenizer.decode(outputs[0], skip_special_tokens=True).strip(' Spørgsmål')
print(input_text, '\n')
print(answer)

Relevante paragraffer: En videoafhøring efter § 340 a eller § 155, stk. 3, kan anvendes som bevis under hovedforhandlingen.
Spørgsmål: Har en person, der er blevet videoafhørt pligt til at afgive forklaring som vidne under hovedforhandlingen?
Indsæt svar her baseret på de relevante paragraffer: 

. Svar: Ja. Svar: Ja. Svar: Ja. Svar: Ja. Svar: Ja. Svar: Ja. Svar: Ja. Svar: Ja. Svar: Ja. Svar: Ja. Svar: Ja. Svar: Ja. Svar: Ja. Svar: Ja. Svar: Ja. Svar: Ja. Svar: Ja. Svar: Ja. Svar: Ja. Svar: Ja. Svar: Ja. Svar: Ja. Svar: Ja. Svar: Ja. Svar: Ja. Svar: Ja. Svar: Ja. Svar: Ja. Svar: Ja. Svar: Ja. Svar: Ja.


In [37]:
print(dev_set['question'][idx])
print(idx)
idx +=1

Har en person, der er blevet videoafhørt pligt til at afgive forklaring som vidne under hovedforhandlingen?
10


In [35]:
idx = 9

In [ ]:
# tjek idx 10